# 🎯 Instruction Optimization

`optimize_instruction()` uses a reward-driven feedback loop to find the
best extraction prompt. It generates structural variations, evaluates each
against a validation sample, and returns the instruction with the highest
field-level F1 score.

In [ ]:
import polars as pl
from pydantic import BaseModel, Field

import loclean

## Define schema and data

We want to extract product information from messy free-text descriptions.
The schema defines the target structure.

In [ ]:
class ProductInfo(BaseModel):
    name: str = Field(..., description="Product name")
    weight_kg: float = Field(..., description="Weight in kilograms")
    price_usd: float = Field(..., description="Price in US dollars")


df = pl.DataFrame(
    {
        "description": [
            "Widget A — 2.5kg, $19.99",
            "Gadget B: weight 500g price $45",
            "Thingamajig, approx 1.2 kilos, 29.50 USD",
            "Widget-A (2500 grams) $19.99",
            "gadget-b ~0.5 kg $45.00",
            "Super Deluxe Widget 3.0kg @ $89.95",
            "Mini Gadget C, 200g, fifteen dollars",
            "Pro Widget X — 4.2 kg — USD 129",
        ]
    }
)

df

## Optimize the extraction instruction

Start from a naive baseline and let the optimizer refine it. The optimizer
samples validation rows, generates prompt variations, scores each against
the schema, and returns the winner.

In [ ]:
best_instruction = loclean.optimize_instruction(
    df,
    target_col="description",
    schema=ProductInfo,
    baseline_instruction="Extract product info.",
    sample_size=6,
)

print("Best instruction found:")
print(best_instruction)

## Use the optimized instruction

Pass the optimized instruction back to `extract()` for production use.

In [ ]:
result = loclean.extract(
    df["description"][0],
    ProductInfo,
    instruction=best_instruction,
)

print(f"Input:  {df['description'][0]}")
print(f"Output: {result}")